## Import Libraries

In [259]:
from numpy.random import seed
seed(888)

import os
import numpy as np
import tensorflow as tf
tf.random.set_seed(404)

from time import strftime
from PIL import Image

## Define Constants

In [2]:
X_TRAIN_PATH = 'MNIST/digit_xtrain.csv'
X_TEST_PATH = 'MNIST/digit_xtest.csv'
Y_TRAIN_PATH = 'MNIST/digit_ytrain.csv'
Y_TEST_PATH = 'MNIST/digit_ytest.csv'

LOGGING_PATH = 'tensorboard_mnist_digit_logs/'
LOGGING_PATH_2 = 'tensorboard_mnist_digit_logs_new/'


NR_CLASSES = 10
VALIDATION_SIZE = 10000

IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_WIDTH*IMAGE_HEIGHT*CHANNELS

## Load Data

In [3]:
%%time

y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int )

CPU times: total: 93.8 ms
Wall time: 91 ms


In [4]:
y_train_all.shape

(60000,)

In [5]:
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)

In [6]:
y_test.shape

(10000,)

In [7]:
%%time

x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int )

CPU times: total: 18.1 s
Wall time: 18.1 s


In [8]:
%%time

x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int )

CPU times: total: 2.98 s
Wall time: 2.98 s


## Explore

In [9]:
x_train_all.shape

(60000, 784)

In [10]:
x_train_all[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

In [11]:
y_train_all.shape

(60000,)

In [12]:
x_test.shape

(10000, 784)

In [13]:
y_train_all[:9]

array([5, 0, 4, 1, 9, 2, 1, 3, 1])

## Data Preprocessing

In [14]:
# Re-scale features
x_train_all, x_test = x_train_all/255.0, x_test/255.0

#### Convert target values to one-hot encoding

In [15]:
values = y_train_all[:5]
np.eye(10)[values]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [16]:
np.eye(10)

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [17]:
y_train_all = np.eye(NR_CLASSES)[y_train_all]

In [18]:
y_train_all.shape

(60000, 10)

In [19]:
y_test = np.eye(NR_CLASSES)[y_test]
y_test.shape

(10000, 10)

## Create Validation dataset from training data

**Challenge:** Split the training dataset into smaller training datatset and a validation dataset for the features and the labels. Create four arrays: x_val, y_val, x_train and y_train from x_train_all and y_train_all. Use validation size of 10000

In [20]:
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

In [21]:
y_train.shape, y_val.shape

((50000, 10), (10000, 10))

## Setup Tensorflow Graph

In [336]:
tf.compat.v1.disable_eager_execution()

X = tf.compat.v1.placeholder(tf.float32, shape=[None, TOTAL_INPUTS], name='X')
Y = tf.compat.v1.placeholder(tf.float32, shape=[None, NR_CLASSES], name='labels')

### Neural Network Architecture
#### Hyperparameters

In [337]:
nr_epochs = 20
learning_rate = 0.001

# Hidden layers
n_hidden1 = 512
n_hidden2 = 64

In [338]:
# with tf.compat.v1.name_scope('First_layer'):


#     initial_w1 = tf.compat.v1.truncated_normal(shape=[TOTAL_INPUTS, n_hidden1], stddev=0.1, seed=42)
#     w1 = tf.compat.v1.Variable(initial_value=initial_w1, name='w1')

#     initial_b1 = tf.constant(value=0.0, shape=[n_hidden1])
#     b1 = tf.compat.v1.Variable(initial_b1, name='b1')

#     layer1_in = tf.matmul(X, w1) + b1
#     layer1_out = tf.nn.relu(layer1_in)

**Challenge:** Set up the second hidden layer. This layer has 64 neurons and needs to work off the outputs of the first hidden layer. Then setup the output layer. Remember, the output layer will use the softmax activation function. 

In [339]:
# with tf.compat.v1.name_scope('second_layer'):

#     initial_w2 = tf.compat.v1.truncated_normal(shape=[n_hidden1, n_hidden2], stddev=0.1, seed=42)
#     w2 = tf.compat.v1.Variable(initial_value=initial_w2, name='w2')

#     initial_b2 = tf.constant(value=0.0, shape=[n_hidden2])
#     b2 = tf.compat.v1.Variable(initial_b2, name='b2')

#     layer2_in = tf.matmul(layer1_out, w2) + b2
#     layer2_out = tf.nn.relu(layer2_in)

In [340]:
# with tf.compat.v1.name_scope('output_layer'):

#     initial_w3 = tf.compat.v1.truncated_normal(shape=[n_hidden2, NR_CLASSES], stddev=0.1, seed=42)
#     w3 = tf.compat.v1.Variable(initial_value=initial_w3, name='w3')

#     initial_b3 = tf.constant(value=0.0, shape=[NR_CLASSES])
#     b3 = tf.compat.v1.Variable(initial_b3, name='b3')

#     layer3_in = tf.matmul(layer2_out, w3) + b3
#     output = tf.nn.softmax(layer3_in)

In [341]:
def setup_layer(input, weight_dim, bias_dim, name):
    
    with tf.compat.v1.name_scope(name):
        initial_w = tf.compat.v1.truncated_normal(shape=weight_dim, stddev=0.1, seed=42)
        w = tf.compat.v1.Variable(initial_value=initial_w, name='W')

        initial_b = tf.constant(value=0.0, shape=bias_dim)
        b = tf.compat.v1.Variable(initial_b, name='B')

        layer_in = tf.matmul(input, w) + b
        
        if name=='out':
            layer_out = tf.nn.softmax(layer_in)
        else:
            layer_out = tf.nn.relu(layer_in)
            
        tf.compat.v1.summary.histogram('weights', w)
        tf.compat.v1.summary.histogram('biases', b)
            
        return layer_out
        

In [342]:
# Model without Dropout
# layer_1 = setup_layer(X, weight_dim=[TOTAL_INPUTS, n_hidden1], 
#                       bias_dim=[n_hidden1], name='layer_1')

# layer_2 = setup_layer(layer_1, weight_dim=[n_hidden1, n_hidden2], 
#                       bias_dim=[n_hidden2], name='layer_2')

# output = setup_layer(layer_2, weight_dim=[n_hidden2, NR_CLASSES], 
#                       bias_dim=[NR_CLASSES], name='out')

# model_name = f'{n_hidden1}-{n_hidden2} LR{learning_rate} E {nr_epochs}'

In [343]:
# Model with Dropout

layer_1 = setup_layer(X, weight_dim=[TOTAL_INPUTS, n_hidden1], 
                      bias_dim=[n_hidden1], name='layer_1')

layer_drop = tf.compat.v1.nn.dropout(layer_1, keep_prob=0.8, name='dropout_layer')

layer_2 = setup_layer(layer_drop, weight_dim=[n_hidden1, n_hidden2], 
                      bias_dim=[n_hidden2], name='layer_2')

output = setup_layer(layer_2, weight_dim=[n_hidden2, NR_CLASSES], 
                      bias_dim=[NR_CLASSES], name='out')

model_name = f'{n_hidden1}-DO-{n_hidden2} LR{learning_rate} E {nr_epochs}'

## Tensorboard Setup

In [344]:
# Folder for Tensorboard

folder_name = f'{model_name} at {strftime("%H %M")}'
directory = os.path.join(LOGGING_PATH_2, folder_name)

try:
    os.makedirs(directory)
except OSError as exception:
    print(exception.strerror)
else:
    print('Succesfully created directories!')

Succesfully created directories!


## Loss, Optimization and Metrics

#### Defining Loss

In [345]:
with tf.compat.v1.name_scope('loss_calc'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=output))

#### Defining Optimizer

In [346]:
with tf.compat.v1.name_scope('optimizer'): 
    optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)
    train_step = optimizer.minimize(loss)

#### Defining the Accuracy metric

In [347]:
with tf.compat.v1.name_scope('accuracy_calc'):
    correct_pred = tf.equal(tf.argmax(output, axis=1), tf.argmax(Y, axis=1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [348]:
with tf.compat.v1.name_scope('performance'):
    tf.compat.v1.summary.scalar('accuracy', accuracy)
    tf.compat.v1.summary.scalar('cost', loss)

#### Check Input Images in Tensorboard

In [349]:
with tf.compat.v1.name_scope('show_image'):
    x_image = tf.reshape(X, [-1, 28, 28, 1])
    tf.compat.v1.summary.image('image_input', x_image, max_outputs=4)

## Run Session

In [350]:
sess = tf.compat.v1.Session()

#### Setup Filewriter and Merge Summaries

In [351]:
merged_summary = tf.compat.v1.summary.merge_all()
# #merged_summary = tf.summary()

train_writer = tf.compat.v1.summary.FileWriter(directory + '/train')
train_writer.add_graph(sess.graph)

validation_writer = tf.compat.v1.summary.FileWriter(directory + '/validation')
validation_writer = tf.compat.v1.summary.FileWriter(directory + '/test')

#### Initialize all the variables


In [352]:
init = tf.compat.v1.global_variables_initializer()
sess.run(init)

In [353]:
#w1.eval(sess)

### Batcing the Data

In [354]:
size_of_batch = 1000

In [355]:
num_examples = y_train.shape[0]
nr_iterations = int(num_examples/size_of_batch)

index_in_epoch = 0

In [356]:
def next_batch(batch_size, data, labels):
    
    global num_examples
    global index_in_epoch
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > num_examples:
        start = 0
        index_in_epoch = batch_size
        
    end = index_in_epoch
    
    return data[start:end], labels[start:end]

In [357]:
for epoch in range(nr_epochs):
    
    ####========== Training Dataset ===========###########
    for i in range(nr_iterations):
        batch_x, batch_y = next_batch(batch_size=size_of_batch, 
                                      data=x_train, labels=y_train)
        feed_dictionary = {X:batch_x, Y:batch_y}
        sess.run(train_step, feed_dict=feed_dictionary)
        
    s, batch_accuracy = sess.run(fetches=[merged_summary,accuracy], feed_dict=feed_dictionary)
    train_writer.add_summary(s, epoch)
        
    print(f'Epoch {epoch} \t| Training Accuracy = {batch_accuracy}')
    
    ####========== Validation Dataset ===========###########
    summary = sess.run(fetches=merged_summary, feed_dict={X:x_val, Y:y_val})
    validation_writer.add_summary(summary, epoch)

print('Done Training!!!')

Epoch 0 	| Training Accuracy = 0.8479999899864197
Epoch 1 	| Training Accuracy = 0.8610000014305115
Epoch 2 	| Training Accuracy = 0.8600000143051147
Epoch 3 	| Training Accuracy = 0.871999979019165
Epoch 4 	| Training Accuracy = 0.8740000128746033
Epoch 5 	| Training Accuracy = 0.8730000257492065
Epoch 6 	| Training Accuracy = 0.9750000238418579
Epoch 7 	| Training Accuracy = 0.9819999933242798
Epoch 8 	| Training Accuracy = 0.984000027179718
Epoch 9 	| Training Accuracy = 0.9860000014305115
Epoch 10 	| Training Accuracy = 0.9829999804496765
Epoch 11 	| Training Accuracy = 0.9860000014305115
Epoch 12 	| Training Accuracy = 0.9860000014305115
Epoch 13 	| Training Accuracy = 0.9879999756813049
Epoch 14 	| Training Accuracy = 0.9890000224113464
Epoch 15 	| Training Accuracy = 0.9879999756813049
Epoch 16 	| Training Accuracy = 0.9909999966621399
Epoch 17 	| Training Accuracy = 0.9900000095367432
Epoch 18 	| Training Accuracy = 0.9900000095367432
Epoch 19 	| Training Accuracy = 0.990000009

In [358]:
# for epoch in range(nr_epochs):
#     for i in range(nr_iterations):
#         batch_x, batch_y = next_batch(batch_size=size_of_batch, 
#                                       data=x_train, labels=y_train)
#         feed_dictionary = {X:batch_x, Y:batch_y}
#         sess.run(train_step, feed_dict=feed_dictionary)
        
#     batch_accuracy = sess.run(fetches=[accuracy], feed_dict=feed_dictionary)
        
#     print(f'Epoch {epoch} \t| Training Accuracy = {batch_accuracy}')
        
# print('Done Training!!!')

## Make a Prediction

In [359]:
img = Image.open('MNIST/test_img.png')
img

In [360]:
## Convert to grey scale using .convert() i.e to black and white
bw = img.convert('L')
bw

In [361]:
img_array = np.invert(bw)
img_array.shape

(28, 28)

In [362]:
## Next we flatten it
test_img = img_array.ravel()
test_img.shape

(784,)

In [363]:
## In fetches argument, we are interested in highest number 
## in that row, hece we use the .argmax() funtion
prediction = sess.run(fetches=tf.argmax(output, axis=1), feed_dict={X:[test_img]})
print(f'Prediction for the test image is {prediction}')

Prediction for the test image is [2]


## Testing and Evaluation

**Challenge:** Calculate the accuracy over the test dataset (x_test and y_test). Use your knowledge of running a session to get the accuracy. Display as a percentage rounded to two decimal numbers.

In [364]:
test_accuracy = sess.run(fetches=accuracy, feed_dict={X:x_test, Y:y_test})
print(f'Test Accuracy = {test_accuracy:0.2%}')

Test Accuracy = 97.19%


In [365]:
x_test.shape

(10000, 784)

## Reset for the Next Run

In [366]:
train_writer.close()
validation_writer.close()
sess.close()
tf.compat.v1.reset_default_graph()